In [9]:
import cv2
import os
import numpy as np
from UNet import UNet
from sklearn.model_selection import train_test_split
from skimage.restoration import denoise_nl_means, estimate_sigma

In [10]:
def leer_imagenes(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [11]:
def convertir_a_grises(images):
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    return gray_images


In [12]:
from skimage.restoration import denoise_nl_means, estimate_sigma

def aplicar_filtro_anisotropico(images):
    filtered_images = []
    for image in images:
        sigma_est = np.mean(estimate_sigma(image))
        denoised = denoise_nl_means(image, h=1.15 * sigma_est, fast_mode=True,
                                    patch_size=5, patch_distance=6)
        filtered_images.append(denoised)
    return filtered_images



In [18]:
def generar_muestras(images, filtered_images, k=32):
    x_list = []
    y_list = []
    for img, filtered_img in zip(images, filtered_images):
        x, y = np.random.randint(0, img.shape[0] - k), np.random.randint(0, img.shape[1] - k)
        original_patch = img[x:x+k, y:y+k]
        filtered_patch = filtered_img[x:x+k, y:y+k]
        x_list.append(original_patch)
        y_list.append(filtered_patch)
    return x_list, y_list


In [19]:
def main(folder='test', k=32):
    images = leer_imagenes(folder)
    gray_images = convertir_a_grises(images)
    filtered_images = aplicar_filtro_anisotropico(gray_images)
    x, y = generar_muestras(gray_images, filtered_images, k=k)
    return x, y

# Ejecutar la función main
x, y  = main()
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)



In [20]:
input_shape = (32, 32, 1)
unet = UNet()
model = unet.build_model(input_shape)


In [21]:
history = model.fit(X_train, epochs=20, validation_data=x_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.